# 人工智能课程25-作业一-垃圾短信分类

# 1.实验介绍

## 1.1 实验背景
垃圾短信 (Spam Messages，SM) 是指未经过用户同意向用户发送不愿接收的商业广告或者不符合法律规范的短信。    
随着手机的普及，垃圾短信在日常生活日益泛滥，已经严重的影响到了人们的正常生活娱乐，乃至社会的稳定。     
据 360 公司 2020 年第一季度有关手机安全的报告提到，360 手机卫士在第一季度共拦截各类垃圾短信约 34.4 亿条，平均每日拦截垃圾短信约 3784.7 万条。      
大数据时代的到来使得大量个人信息数据得以沉淀和积累，但是庞大的数据量缺乏有效的整理规范；   
在面对量级如此巨大的短信数据时，为了保证更良好的用户体验，如何从数据中挖掘出更多有意义的信息为人们免受垃圾短信骚扰成为当前亟待解决的问题。

## 1.2 实验要求
1) 任务提供包括数据读取、基础模型、模型训练等基本代码  
2) 需完成核心模型构建代码，并将模型在测试数据集上的预测结果下载到本地。  


## 1.3 实验环境 
可以使用基于 Python 的 Pandas、Numpy、Sklearn 等库进行相关特征处理，使用 Paddle 框架训练分类器，使用过程中请注意 Python 包（库）的版本。

## 1.4 作业提交
1. 将实验二和实验三Todo部分的代码、实验四的完整代码放到todo.py文件进行提交。
2. 将最终在私有测试集上的预测结果'predictions.csv'，下载保存到本地文件predictions.csv，进行提交。

## 1.5 参考资料
- Numpy：https://www.numpy.org/
- Pandas: https://pandas.pydata.org/
- Sklearn: https://scikit-learn.org/
- jieba: https://github.com/fxsjy/jieba
- 四川大学机器智能实验室停用词库：https://github.com/goto456/stopwords/blob/master/scu_stopwords.txt

# 2.实验内容

## 2.1 数据集
- 该数据集包括了约 1 万条数据，有 3 个字段 label、 message 和 msg_new， 分别代表了短信的类别、短信的内容和分词后的短信
- 中文分词工具 [jieba](https://github.com/fxsjy/jieba)
- 0 代表正常的短信，1 代表恶意的短信
- 正常短信和恶意短信举例：

|label|message（短信内容）|msg_new（短信分词后）|
|--|--|--|
|0|人们经常是失去了才发现它的珍贵|人们 经常 是 失去 了 才 发现 它 的 珍贵|
|1|本人现在承办驾驶证业务!招收学员，一对 一教学|本人 现在 承办 驾驶证 业务 ! 招收 学员 ， 一对   一 教学|

In [ ]:
# 导入相关的包
import paddle
import paddle.nn.functional as F
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

In [ ]:
# 解压数据集
import os
import zipfile

if not os.path.exists("./sms_v4"):
    zf = zipfile.ZipFile('./sms_v4.zip','r')
    # Extract a member from the archive to the current working directory
    for f in zf.namelist():
        zf.extract(f, './data/sms_v4')  # 循环解压，将文件解压到指定路径
    zf.close()

In [ ]:
# 数据集的路径
data_path = "./data/sms_v4/sms_pub.csv"
# 读取数据
sms = pd.read_csv(data_path, encoding='utf-8')
# 显示前 5 条数据
sms.head()
# 私有测试集的路径
data_path = "./data/sms_v4/sms_private.csv"
# 读取私有测试集数据
sms_private = pd.read_csv(data_path, encoding='utf-8')

In [ ]:
# 显示数据集的一些信息
sms.groupby('label').describe()

## 2.2 数据处理

### 2.2.1 停用词

停用词是指在信息检索中，为节省存储空间和提高搜索效率，在处理自然语言数据（或文本）之前或之后会自动过滤掉某些字或词，这些字或词即被称为 Stop Words（停用词）。      
这些停用词都是人工输入、非自动化生成的，生成后的停用词会形成一个停用词库。        
本次作业中采用的是[四川大学机器智能实验室停用词库](https://github.com/goto456/stopwords/blob/master/scu_stopwords.txt)

In [ ]:
def read_stopwords(stopwords_path):
    """
    读取停用词库
    :param stopwords_path: 停用词库的路径
    :return: 停用词列表
    """
    with open(stopwords_path, 'r', encoding='utf-8') as f:
        stopwords = f.read()
    stopwords = stopwords.splitlines()
    return stopwords

In [ ]:
# 停用词库路径
stopwords_path = r'./data/sms_v4/scu_stopwords.txt'
# 读取停用词
stopwords = read_stopwords(stopwords_path)
# 展示一些停用词
print(stopwords[-20:])

### 2.2.2 文本向量化的方法

**1. CountVectorizer**  
目前拥有的数据是长度不统一的文本数据，而绝大多数机器学习算法需要的输入是向量，因此文本类型的数据需要经过处理得到向量。    
我们可以借助 sklearn 中 **CountVectorizer** 来实现文本的向量化，CountVectorizer 实际上是在统计**每个词出现的次数**，这样的模型也叫做**词袋模型**。

In [ ]:
# 假如我们有这样三条短信
simple_train = ['call you tonight', 'Call me a cab', 'Please call me... PLEASE!']

# 导入 CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

# 从训练数据中学习词汇表
vect.fit(simple_train)

# 查看学习到的词汇表
vect.get_feature_names_out()

In [ ]:
# 将训练数据向量化，得到一个矩阵
simple_train_dtm = vect.transform(simple_train)
# 由于该矩阵的维度可能十分大，而其中大部分都为 0，所以会采用稀疏矩阵来存储
simple_train_dtm

In [ ]:
# 将稀疏矩阵转为一般矩阵查看里面的内容
simple_train_dtm.toarray()

In [ ]:
# 结合词汇表和转为得到的矩阵来直观查看内容
pd.DataFrame(simple_train_dtm.toarray(), columns=vect.get_feature_names_out())

**2. TfidfVectorizer**  
与 CountVectorizer 类似的还有 TfidfVectorizer 。        
TF-IDF 算法是创建在这样一个假设之上的：                     
对区别文档最有意义的词语应该是那些在文档中出现频率高的词语，因此选择特征空间坐标系取 TF 词频作为测度，就可以体现同类文本的特点。                                    
另外考虑到单词区别不同类别的能力，TF-IDF 法认为一个单词出现的文本频数越小，它区别不同类别文本的能力就越大。     
因此引入了逆文本频度 IDF 的概念，以 TF 和 IDF 的乘积作为特征空间坐标系的取值测度，并用它完成对权值 TF 的调整，调整权值的目的在于突出重要单词，抑制次要单词。    
在本质上 IDF 是一种试图抑制噪声的加权，并且单纯地认为文本频率小的单词就越重要，文本频率大的单词就越无用。    
 
其中 TF、 IDF 和 TF-IDF 的含义如下：
+ TF：词频。
$$TF(w) = \frac{词 w 在文档中出现的次数}{文档的总词数}$$
+ IDF：逆向文件频率。有些词可能在文本中频繁出现，但并不重要，也即信息量小，如 is, of, that 这些单词，这些单词在语料库中出现的频率也非常大，我们就可以利用这点，降低其权重。
$$IDF(w) = ln \frac{语料库的总文档数}{语料库中词 w 出现的文档数}$$
+ TF-ID 综合参数：TF - IDF = TF * IDF

In [ ]:
# 导入 TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
# 在训练数据上拟合并将其转为为 tfidf 的稀疏矩阵形式
simple_train_dtm = tfidf.fit_transform(simple_train)
# 将稀疏矩阵转为一般矩阵
simple_train_dtm.toarray()
# 结合词汇表和转为得到的矩阵来直观查看内容
pd.DataFrame(simple_train_dtm.toarray(), columns=tfidf.get_feature_names_out())

### 2.2.3 划分训练集和测试集

一般的数据集会划分为两个部分：
+ 训练数据：用于训练，构建模型
+ 测试数据：在模型检验时使用，用于评估模型是否有效
<br>

这里划分比例：7:3

<br>

`sklearn.model_selection.train_test_split(x, y, test_size, random_state )`
   +  `x`：数据集的特征值
   +  `y`： 数据集的标签值
   +  `test_size`： 如果是浮点数，表示测试集样本占比；如果是整数，表示测试集样本的数量。
   +  `random_state`： 随机数种子,不同的种子会造成不同的随机采样结果。相同的种子采样结果相同。
   +  `return` 训练集的特征值 `x_train` 测试集的特征值 `x_test` 训练集的目标值 `y_train` 测试集的目标值 `y_test`。

In [ ]:
# 构建训练集和测试集
from sklearn.model_selection import train_test_split
X = np.array(sms.msg_new)
y = np.array(sms.label)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)
print("总共的数据大小", X.shape)
print("训练集数据大小", X_train.shape)
print("测试集数据大小", X_test.shape)
# 构建私有测试集
X_private = np.array(sms_private.msg_new)
y_private = np.array(sms_private.label)

**注意：** CountVectorizer 默认会去除长度为 1 的字符串，这会丢失一部分信息，通过将 token_pattern 的属性值改为正则表达式 (?u)\b\w+\b 可以解决这个问题。

In [ ]:
# 以 CountVectorizer 为例将数据集向量化
from sklearn.feature_extraction.text import CountVectorizer
# 设置匹配的正则表达式和停用词
vect = CountVectorizer(token_pattern=r"(?u)\b\w+\b", stop_words=stopwords)
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)
X_private_dtm = vect.transform(X_private)

词袋模型得到的输入特征在两万维以上，为了提高模型训练效果，使用卡方检验选择最具代表性的特征，进行降维。

In [ ]:
# 使用卡方检验选择最具代表性的1000个特征
from sklearn.feature_selection import SelectKBest, chi2
selector = SelectKBest(chi2, k=1000)
X_train_selected = selector.fit_transform(X_train_dtm, y_train)
X_test_selected = selector.transform(X_test_dtm)
X_private_selected = selector.transform(X_private_dtm)

# 3.实验题目

**题目内容：** 根据一段中文文本（ 200 个中文字符以内），预测这段文本是否为垃圾短信。 

## 3.1 实验要求

实验要求：
1. 实验一：使用线性回归和设定阈值直接进行分类，通过梯度下降求解，观察结果。
2. 实验二：在实验一的基础上增加正则化项（L1，L2），记录结果，并与实验一的结果进行对比。
3. 实验三：使用对数几率回归进行分类，通过梯度下降求解，记录结果，并与实验一、二的结果进行对比。
4. 实验四：自行尝试sklearn库中的Logistic Regression或其他模型，调整参数进行训练，记录结果。

提交内容：
1. 将实验二和实验三Todo部分的代码、实验四的完整代码放到todo.py文件进行提交。
2. 将最终在私有测试集上的预测结果'predictions.csv'，下载保存到本地文件predictions.csv，进行提交。


## 3.2 模型的搭建和训练

### 3.2.1 实验一

In [ ]:
# ----------------- 导入相关的库 -----------------
from sklearn import metrics

# 自定义线性分类器（实际为线性回归结构）
class LinearClassifier(paddle.nn.Layer):
    def __init__(self, input_dim):
        super().__init__()
        self.linear = paddle.nn.Linear(input_dim, 1)  # 单输出线性层（无激活函数）

    def forward(self, x):
        return self.linear(x)  # 直接输出线性结果，不使用sigmoid激活函数

# 模型训练函数
def train_1(X_train_dtm, y_train):
    # 参数设置
    EPOCHS = 300         # 总训练轮次
    BATCH_SIZE = 64      # 批大小
    LEARNING_RATE = 0.1  # 学习率
    MODEL_PATH = "linear_classifier.pdparams"  # 模型保存路径
    
    # 转换数据格式：稀疏矩阵转密集数组 + 类型转换
    X_train = X_train_dtm.toarray().astype('float32')
    y_train = y_train.reshape(-1, 1).astype('float32')
    
    # 初始化模型
    model = LinearClassifier(input_dim=X_train.shape[1])
    optimizer = paddle.optimizer.SGD(  # 选择随机梯度下降优化器
        learning_rate=LEARNING_RATE,
        parameters=model.parameters()
    )
    
    best_acc = 0.0
    for epoch in range(EPOCHS):
        # 训练过程（批训练）
        epoch_loss = 0
        for i in range(0, len(X_train), BATCH_SIZE):
            # 数据分批次加载（避免一次性加载大矩阵）
            batch_X = X_train[i:i+BATCH_SIZE]
            batch_y = y_train[i:i+BATCH_SIZE]
            
            # 转换为Paddle Tensor
            X_tensor = paddle.to_tensor(batch_X)
            y_tensor = paddle.to_tensor(batch_y)

            # 前向计算与损失计算（使用MSE）
            outputs = model(X_tensor)
            loss = paddle.nn.functional.mse_loss(outputs, y_tensor)  # 回归损失
            
            # 反向传播与参数更新
            loss.backward()
            optimizer.step()
            optimizer.clear_grad()  # 清除梯度
            epoch_loss += loss.numpy()

        # 每个epoch后评估并保存最佳模型
        with paddle.no_grad():  # 关闭梯度计算
            train_pred = model(paddle.to_tensor(X_train)).numpy()
            # 通过阈值0.5将回归输出转为分类结果
            current_acc = np.mean((train_pred > 0.5).astype(int) == y_train)
            
            if current_acc > best_acc:
                best_acc = current_acc
                paddle.save(model.state_dict(), MODEL_PATH)  # 保存模型参数
        
        print(f"Epoch {epoch+1}/{EPOCHS} Loss: {epoch_loss:.4f} | Accuracy: {current_acc:.4f}")

    print(f"训练完成，模型已保存至：{MODEL_PATH}")
    return MODEL_PATH

# 模型评估函数
def evaluate_1(model_path, X_test_dtm, y_test):
    # 数据预处理（与训练集一致）
    X_test = X_test_dtm.toarray().astype('float32')
    y_test = y_test.reshape(-1, 1).astype('int')  # 转换为整型标签
    
    # 初始化模型结构（与训练时一致）
    model = LinearClassifier(input_dim=X_test.shape[1])
    model.set_state_dict(paddle.load(model_path))  # 加载保存的模型参数
    model.eval()  # 设置评估模式（关闭dropout等训练专用层）
    
    # 进行预测
    with paddle.no_grad():
        test_outputs = model(paddle.to_tensor(X_test)).numpy()
        y_pred = (test_outputs > 0.5).astype(int)
    
    return y_pred

# ----------------- 执行训练与评估 -----------------
model_path = train_1(X_train_selected, y_train)
y_pred = evaluate_1(model_path, X_test_selected, y_test)
# 生成评估报告
print("\n在测试集上的混淆矩阵：")
print(metrics.confusion_matrix(y_test, y_pred))

print("\n在测试集上的分类结果报告：")
print(metrics.classification_report(y_test, y_pred))

print("在测试集上的 f1-score：")
print(metrics.f1_score(y_test, y_pred))

### 3.2.2 实验二

In [ ]:
# ----------------- 导入相关的库 -----------------
from sklearn import metrics

# 模型训练函数
def train_2(X_train_dtm, y_train, reg_type='l1', lambda_=0.01):
    # 参数设置
    EPOCHS = 300
    BATCH_SIZE = 64
    LEARNING_RATE = 0.1
    MODEL_PATH = "linear_classifier_with_regularization.pdparams"
    
    # 转换数据格式
    X_train = X_train_dtm.toarray().astype('float32')
    y_train = y_train.reshape(-1, 1).astype('float32')
    
    # 初始化模型（依然使用线性回归结构）
    model = LinearClassifier(input_dim=X_train.shape[1])
    optimizer = paddle.optimizer.SGD( # 使用随机梯度下降优化器
        learning_rate=LEARNING_RATE,
        parameters=model.parameters()
    )
    
    best_acc = 0.0
    for epoch in range(EPOCHS):
        # 训练过程
        epoch_loss = 0
        for i in range(0, len(X_train), BATCH_SIZE):
            batch_X = X_train[i:i+BATCH_SIZE]
            batch_y = y_train[i:i+BATCH_SIZE]
            
            X_tensor = paddle.to_tensor(batch_X)
            y_tensor = paddle.to_tensor(batch_y)

            outputs = model(X_tensor)
            mse_loss = paddle.nn.functional.mse_loss(outputs, y_tensor)
            
            # 根据reg_type(l1, l2)和lambda计算正则项损失reg_loss（通常不对偏置项正则化）
            # --------------------------- TODO ---------------------------------------
            # ------------------------------------------------------------------------
            
            total_loss = mse_loss + reg_loss
            
            # 反向传播
            total_loss.backward()
            optimizer.step()
            optimizer.clear_grad()
            epoch_loss += total_loss.numpy()[0]

        # 每个epoch后保存最佳模型
        with paddle.no_grad():
            train_pred = model(paddle.to_tensor(X_train)).numpy()
            current_acc = np.mean((train_pred > 0.5).astype(int) == y_train)
            
            if current_acc > best_acc:
                best_acc = current_acc
                paddle.save(model.state_dict(), MODEL_PATH)
        
        print(f"Epoch {epoch+1}/{EPOCHS} Loss: {epoch_loss:.4f} | Accuracy: {current_acc:.4f}")

    print(f"训练完成，模型已保存至：{MODEL_PATH}")
    return MODEL_PATH

# 模型评估函数
def evaluate_2(model_path, X_test_dtm, y_test):
    X_test = X_test_dtm.toarray().astype('float32')
    y_test = y_test.reshape(-1, 1).astype('int')
    
    # 初始化模型结构
    model = LinearClassifier(input_dim=X_test.shape[1])
    model.set_state_dict(paddle.load(model_path))
    model.eval()
    
    # 进行预测
    with paddle.no_grad():
        test_outputs = model(paddle.to_tensor(X_test)).numpy()
        y_pred = (test_outputs > 0.5).astype(int)
    
    return y_pred

# 训练模型（正则化项和权重自行尝试）
model_path = train_2(X_train_selected, y_train, reg_type='l2', lambda_=0.01)

# 评估模型
y_pred = evaluate_2(model_path, X_test_selected, y_test)

# 生成评估报告
print("\n在测试集上的混淆矩阵：")
print(metrics.confusion_matrix(y_test, y_pred))

print("\n在测试集上的分类结果报告：")
print(metrics.classification_report(y_test, y_pred))

print("在测试集上的 f1-score：")
print(metrics.f1_score(y_test, y_pred))

### 3.2.3 实验三

In [ ]:
# ----------------- 导入相关的库 -----------------
from sklearn import metrics

# 逻辑回归分类器
class LogisticClassifier(paddle.nn.Layer):
    def __init__(self, input_dim):
        super().__init__()
        self.linear = paddle.nn.Linear(input_dim, 1)  # 线性变换层
        self.sigmoid = paddle.nn.Sigmoid()  # 将线性输出转为概率的激活函数

    def forward(self, x):
        # 将线性结果通过sigmoid得到概率值
        # --------------------------- TODO ---------------------------------------
        # ------------------------------------------------------------------------

# 模型训练函数
def train_3(X_train_dtm, y_train):
    # 参数设置
    EPOCHS = 300
    BATCH_SIZE = 64
    LEARNING_RATE = 0.1 
    MODEL_PATH = "logistic_classifier.pdparams"

    # 数据预处理
    X_train = X_train_dtm.toarray().astype('float32')
    y_train = y_train.reshape(-1, 1).astype('float32')

    # 模型初始化
    model = LogisticClassifier(input_dim=X_train.shape[1])
    optimizer = paddle.optimizer.SGD(  # 使用随机梯度下降优化器
        learning_rate=LEARNING_RATE,
        parameters=model.parameters()
    )
    
    best_acc = 0.0
    for epoch in range(EPOCHS):
        epoch_loss = 0 
        for i in range(0, len(X_train), BATCH_SIZE):
            # 数据批次划分
            batch_X = X_train[i:i+BATCH_SIZE]
            batch_y = y_train[i:i+BATCH_SIZE]
            
            X_tensor = paddle.to_tensor(batch_X)
            y_tensor = paddle.to_tensor(batch_y)

            # 使用二元交叉熵损失
            # --------------------------- TODO ---------------------------------------
            # ------------------------------------------------------------------------
            
            # 反向传播与参数更新
            loss.backward()
            optimizer.step()
            optimizer.clear_grad()
            epoch_loss += loss.numpy().item()

        # 模型验证与保存
        with paddle.no_grad():
            y_pred = model(paddle.to_tensor(X_train)).numpy()
            current_acc = np.mean((y_pred > 0.5).astype(int) == y_train)
            
            if current_acc > best_acc:
                best_acc = current_acc
                paddle.save(model.state_dict(), MODEL_PATH)
        
        print(f"Epoch {epoch+1}/{EPOCHS} Loss: {epoch_loss:.4f} | Accuracy: {current_acc:.4f}")

    print(f"训练完成，模型已保存至：{MODEL_PATH}")
    return MODEL_PATH

# 模型评估函数
def evaluate_3(model_path, X_test_dtm, y_test):
    # 数据预处理
    X_test = X_test_dtm.toarray().astype('float32')
    y_test = y_test.reshape(-1, 1).astype('int')

    # 模型加载
    model = LogisticClassifier(input_dim=X_test.shape[1])
    model.set_state_dict(paddle.load(model_path))
    model.eval()

    # 预测执行
    with paddle.no_grad():
        y_pred = (model(paddle.to_tensor(X_test)).numpy() > 0.5).astype(int)
    
    return y_pred

# ----------------- 执行训练与评估 -----------------
model_path = train_3(X_train_selected, y_train)  # 启动训练
y_pred = evaluate_3(model_path, X_test_selected, y_test)  # 执行评估
# 评估指标输出
print("\n在测试集上的混淆矩阵：")
print(metrics.confusion_matrix(y_test, y_pred))

print("\n在测试集上的分类结果报告：")
print(metrics.classification_report(y_test, y_pred))

print("在测试集上的 f1-score：")
print(metrics.f1_score(y_test, y_pred))

### 3.2.4 实验四

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import joblib

def train_4(X_train_dtm, y_train):
    # 自行探索sklearn库里的Logistic Regression或别的模型，观察效果如何
    model = LogisticRegression(max_iter=300, solver='lbfgs')  
    model.fit(X_train_dtm, y_train)  # 训练模型
    model_path = "sklearn.pkl"
    joblib.dump(model, model_path)  # 保存模型
    print(f"训练完成，模型已保存至：{model_path}")
    
    return model_path

def evaluate_4(model_path, X_test_dtm, y_test):
    model = joblib.load(model_path)  # 加载模型
    y_pred = model.predict(X_test_dtm)  # 预测

    return y_pred

# 训练和评估
model_path = train_4(X_train_selected, y_train)
y_pred = evaluate_4(model_path, X_test_selected, y_test)
# 评估指标输出
print("\n在测试集上的混淆矩阵：")
print(metrics.confusion_matrix(y_test, y_pred))

print("\n在测试集上的分类结果报告：")
print(metrics.classification_report(y_test, y_pred))

print("在测试集上的 f1-score：")
print(metrics.f1_score(y_test, y_pred))

## 3.3 模型的预测

1. 请加载你认为训练最佳的模型，即请按要求选择selected_model。
2. 将最终在测试数据集上的预测结果'predictions.csv'，下载到本地，进行提交。

In [ ]:
# 将真实标签与预测结果导出为CSV文件
def export_predictions(y_true, y_pred, filename='predictions.csv'):
    df = pd.DataFrame({
        'True_Label': y_true.flatten(),
        'Predicted_Label': y_pred.flatten()
    })
    df.to_csv(filename, index=False)
    print(f"预测结果已保存至：{filename}")

# 根据模型类型选择并执行预测
def select_and_predict(model_type, X_test, y_test):
    """
    参数：
        model_type - 模型类型字符串 ('linear', 'regularized', 'logistic', 'sklearn')
        X_test - 测试集特征矩阵
        y_test - 测试集标签
    返回：
        预测结果数组
    """
    # 模型路径映射
    model_paths = {
        'linear': 'linear_classifier.pdparams',
        'regularized': 'linear_classifier_with_regularization.pdparams',
        'logistic': 'logistic_classifier.pdparams',
        'sklearn': 'sklearn.pkl'
    }
    
    # 根据类型选择评估函数
    if model_type == 'logistic':
        y_pred = evaluate_3(model_paths[model_type], X_test, y_test)
    elif model_type == 'sklearn':
        y_pred = evaluate_4(model_paths[model_type], X_test, y_test)
    else:
        y_pred = evaluate_2(model_paths[model_type], X_test, y_test) if model_type == 'regularized' else evaluate_1(model_paths[model_type], X_test, y_test)
    
    return y_pred

    
selected_model = 'logistic'  # 在linear, regularized, logistic和sklearn中选择
predictions = select_and_predict(selected_model, X_private_selected, y_private)

# 导出结果
export_predictions(y_private, predictions, 'predictions.csv')